# Local tests

In [1]:
import os
from datetime import datetime
from sqlalchemy import create_engine, text
from sqlalchemy.orm import sessionmaker, joinedload

from app.models.orders import (
    OrderItems,
    OrderStatus,
    Orders,
    CurrentOrderStatus,
    DeliveryOptions,
    DeliveryStatus,
    ShippingDetails,
)

from app.models.products import (  # Assuming your models are in an 'app.models' package
    Articles,
    Brand,
    Categories,
    Colors,
    ProductCategories,
    ProductOccasionalCategories,
    Products,
    SizeSystems,
    Sizing,
    StockKeepingUnits,
    Types,
    Variants,
    ProductCurrentStatus,
    ProductStatus,
    OwnerType,
    Condition,
    ArticleStatus,
    ArticleCurrentStatus,
    TypesOfOperations,
)
from app.models.users import (
    UserAddresses,
    UserPhotos,
    Users,
)
from app.schemas.users import AddressType

In [2]:
# Database Connection (using tunneling)
DATABASE_URL = os.getenv("DATABASE_URL")  # Get your database URL
engine = create_engine(DATABASE_URL)
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)

In [3]:
def test_database_connection():
    """Tests the database connection."""
    try:
        with SessionLocal() as db:
            db.execute(text("SELECT 1"))  # Simple query to test connection
            print("Database connection successful!")
    except Exception as e:
        print(f"Database connection failed: {e}")


def test_product_crud():
    """Tests CRUD operations for Products."""
    with SessionLocal() as db:
        # CREATE
        try:
            # Create necessary related objects
            brand = Brand(name="Test Brand")
            category = Categories(name="Test Category", is_default=False)
            color = Colors(name="Test Color")
            size_system = SizeSystems(name="Test Size System")
            product_status = ProductStatus(name=ProductCurrentStatus.Available)

            db.add_all([brand, category, color, size_system, product_status])
            db.commit()

            product = Products(
                name="Test Product",
                brand_id=brand.id,
                description="This is a test product.",
                status_code=product_status.id,
            )

            sku = StockKeepingUnits(sku_name="TESTSKU", free_articles_count=10)
            db.add(sku)
            db.commit()

            variant = Variants(
                name="Test Variant",
                product_id=product.id,
                color_id=color.id,
                sku_id=sku.id,
            )
            product.variants.append(variant)

            db.add_all([product, variant])
            db.commit()
            db.refresh(product)

            product_category = ProductCategories(
                product_id=product.id, category_id=category.id
            )
            db.add(product_category)
            db.commit()

            sizing = Sizing(
                label="Test Size",
                variant_id=variant.id,
                size_system_id=size_system.id,
            )
            db.add(sizing)
            db.commit()

            print(f"Created product with ID: {product.id}")

        except Exception as e:
            print(f"Error creating product: {e}")
            db.rollback()
            return

        # READ
        try:
            read_product = db.query(Products).filter_by(id=product.id).first()
            assert read_product is not None
            assert read_product.name == "Test Product"
            print("Read product successfully.")
        except Exception as e:
            print(f"Error reading product: {e}")
            return

        # UPDATE
        try:
            read_product.description = "Updated product description"
            db.commit()
            db.refresh(read_product)
            assert read_product.description == "Updated product description"
            print("Updated product successfully.")
        except Exception as e:
            print(f"Error updating product: {e}")
            return

        # DELETE
        try:
            db.delete(read_product)
            db.commit()
            deleted_product = db.query(Products).filter_by(id=product.id).first()
            assert deleted_product is None
            print("Deleted product successfully.")
        except Exception as e:
            print(f"Error deleting product: {e}")
            return


def test_order_crud():
    """Tests CRUD operations for Orders."""
    with SessionLocal() as db:
        # CREATE
        try:
            user = Users(login="testuser", password="testpassword")
            db.add(user)
            db.commit()

            user_address = UserAddresses(
                address_line1="Test Address",
                city="Test City",
                country="Test Country",
                postal_code="12345",
                user_id=user.id,
                address_type=AddressType.Both,
            )
            db.add(user_address)
            db.commit()

            order_status = OrderStatus(name=CurrentOrderStatus.Placed)
            db.add(order_status)
            db.commit()

            order = Orders(
                user_id=user.id,
                status_code=order_status.id,
                total_price=100.00,
                total_delivery_price=10.00,
            )
            db.add(order)
            db.commit()
            db.refresh(order)

            print(f"Created order with ID: {order.id}")

        except Exception as e:
            print(f"Error creating order: {e}")
            db.rollback()
            return

        # READ
        try:
            read_order = db.query(Orders).filter_by(id=order.id).first()
            assert read_order is not None
            assert read_order.total_price == 100.00
            print("Read order successfully.")
        except Exception as e:
            print(f"Error reading order: {e}")
            return

        # UPDATE
        try:
            read_order.total_price = 120.00
            db.commit()
            db.refresh(read_order)
            assert read_order.total_price == 120.00
            print("Updated order successfully.")
        except Exception as e:
            print(f"Error updating order: {e}")
            return

        # DELETE
        try:
            db.delete(read_order)
            db.commit()
            deleted_order = db.query(Orders).filter_by(id=order.id).first()
            assert deleted_order is None
            print("Deleted order successfully.")
        except Exception as e:
            print(f"Error deleting order: {e}")
            return


def test_order_item_crud():
    """Tests CRUD operations for OrderItems (requires a Product and Order to exist)."""
    with SessionLocal() as db:
        # CREATE
        try:
            # Make sure a Product and Order exist
            test_product_crud()  # This will create a test product
            test_order_crud()  # This will create a test order

            product = db.query(Products).filter_by(name="Test Product").first()
            order = db.query(Orders).options(joinedload(Orders.users)).first()

            if not product or not order:
                print("Product or Order not found, skipping OrderItem CRUD test.")
                return

            # Create a DeliveryOptions instance (required for ShippingDetails)
            delivery_option = DeliveryOptions(name="Standard Shipping", cost=5.00)
            article_status = ArticleStatus(name=ArticleCurrentStatus.Available)
            types_of_operation = TypesOfOperations(name="Rental")
            db.add_all([delivery_option, article_status, types_of_operation])
            db.commit()

            # Create a ShippingDetails instance
            shipping_details = ShippingDetails(
                tracking_number="1234567890",
                actual_delivery_date=datetime.now(),
                delivery_option_id=delivery_option.id,
                user_address_id=order.users.addresses[0].id,
                delivery_status=DeliveryStatus.Pending,
                is_peer_to_peer=False,
            )
            db.add(shipping_details)
            db.commit()

            article = Articles(
                article="Test Article",
                owner_type=OwnerType.Platform,
                condition=Condition.New,
                sku_id=product.variants[0].sku_id,
                status_code=article_status.id,
                types_of_operation_id=types_of_operation.id,
            )
            db.add(article)
            db.commit()

            order_item = OrderItems(
                order_id=order.id,
                article_id=article.id,
                shipping_id=shipping_details.id,
                start_date="2024-03-10",
                end_date="2024-03-15",
                time_start="10:00",
                price=10000,
            )
            db.add(order_item)
            db.commit()
            db.refresh(order_item)

            print(f"Created OrderItem with ID: {order_item.id}")

        except Exception as e:
            print(f"Error creating OrderItem: {e}")
            db.rollback()
            return

        # READ
        try:
            read_order_item = db.query(OrderItems).filter_by(id=order_item.id).first()
            assert read_order_item is not None
            assert read_order_item.order_id == order.id
            print("Read OrderItem successfully.")
        except Exception as e:
            print(f"Error reading OrderItem: {e}")
            return

        # UPDATE
        try:
            read_order_item.start_date = "2024-03-12"
            db.commit()
            db.refresh(read_order_item)
            assert read_order_item.start_date == "2024-03-12"
            print("Updated OrderItem successfully.")
        except Exception as e:
            print(f"Error updating OrderItem: {e}")
            return

        # DELETE
        try:
            db.delete(read_order_item)
            db.commit()
            deleted_order_item = (
                db.query(OrderItems).filter_by(id=order_item.id).first()
            )
            assert deleted_order_item is None
            print("Deleted OrderItem successfully.")
        except Exception as e:
            print(f"Error deleting OrderItem: {e}")
            return

In [4]:
if __name__ == "__main__":
    # test_database_connection()
    # test_product_crud()
    # test_order_crud()
    test_order_item_crud()

Created product with ID: af51d8f1-a8d8-48de-968d-cfb90fc56d65
Read product successfully.
Updated product successfully.
Deleted product successfully.
Created order with ID: 8020b22a-581f-4062-8919-d22675670611
Read order successfully.
Updated order successfully.
Deleted order successfully.
Error creating OrderItem: (pg8000.dbapi.ProgrammingError) {'S': 'ERROR', 'V': 'ERROR', 'C': '23503', 'M': 'insert or update on table "order_items" violates foreign key constraint "fk_article_to_order_items"', 'D': 'Key (article_id)=(4af1a6c2-607d-4133-8f22-a197240a6b77) is not present in table "articles".', 's': 'public', 't': 'order_items', 'n': 'fk_article_to_order_items', 'F': 'ri_triggers.c', 'L': '2596', 'R': 'ri_ReportViolation'}
[SQL: INSERT INTO order_items (start_date, end_date, time_start, price, delivery_price, comment, order_id, article_id, shipping_id, updated_at, deleted_at) VALUES (%s::TIMESTAMP WITHOUT TIME ZONE, %s::TIMESTAMP WITHOUT TIME ZONE, %s::TIME WITHOUT TIME ZONE, %s::NUMERIC,